In [1]:
import os

# set current file as the working directory
# os.chdir(os.path.dirname(os.path.abspath(__file__)))

from scipy import stats

import tensorflow as tf

# from model import Model
import numpy as np
from keras.utils import to_categorical

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import argparse
import pickle
# from helper.stats import stats_dataset
# from helper.utils import pickle_store_dataset,load_from_pickle
import time 
import tensorflow as tf
import concurrent.futures
import torch
import sys
from vit_pytorch import SimpleViT

2024-03-31 16:27:01.992368: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-31 16:27:02.044760: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-31 16:27:03.026109: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
data_dir = './data'
def load_pickle(file_name):
    with open(f'{data_dir}/{file_name}', 'rb') as f:
        return pickle.load(f)

# Use concurrent.futures to load the data in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=70) as executor:
    train_data_future = executor.submit(load_pickle, 'train_data.pkl')
    train_labels_future = executor.submit(load_pickle, 'train_labels.pkl')
    # test_data_future = executor.submit(load_pickle, 'test_data.pkl')
    # test_labels_future = executor.submit(load_pickle, 'test_labels.pkl')
    val_data_future = executor.submit(load_pickle, 'val_data.pkl')
    val_labels_future = executor.submit(load_pickle, 'val_labels.pkl')

    # Wait for all futures to complete
    train_data = train_data_future.result()
    train_labels = train_labels_future.result()
    # test_data = test_data_future.result()
    # test_labels = test_labels_future.result()
    val_data = val_data_future.result()
    val_labels = val_labels_future.result()

In [3]:
type(train_data)

numpy.ndarray

In [4]:
v = SimpleViT(
    image_size = 256,
    patch_size = 128,
    num_classes = 43,
    dim = 32,
    depth = 6,
    heads = 16,
    mlp_dim = 2048
)


v.train(True)

SimpleViT(
  (to_patch_embedding): Sequential(
    (0): Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=128, p2=128)
    (1): LayerNorm((49152,), eps=1e-05, elementwise_affine=True)
    (2): Linear(in_features=49152, out_features=32, bias=True)
    (3): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  )
  (transformer): Transformer(
    (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (layers): ModuleList(
      (0-5): 6 x ModuleList(
        (0): Attention(
          (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
          (attend): Softmax(dim=-1)
          (to_qkv): Linear(in_features=32, out_features=3072, bias=False)
          (to_out): Linear(in_features=1024, out_features=32, bias=False)
        )
        (1): FeedForward(
          (net): Sequential(
            (0): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
            (1): Linear(in_features=32, out_features=2048, bias=True)
            (2): GELU(approximate='none')
      

In [5]:
train_data = np.transpose(train_data, (0, 3, 1, 2))

In [6]:
train_data.shape

(61201, 3, 256, 256)

In [7]:
# train_tensor = tf.convert_to_tensor(train_data)
train_tensor = torch.from_numpy(train_data)
train_tensor = train_tensor.float()

In [8]:
train_tensor.shape

torch.Size([61201, 3, 256, 256])

In [9]:
type(train_tensor)

torch.Tensor

In [10]:
train_tensor.device

device(type='cpu')

In [17]:
v(train_tensor)